In [ ]:
import random
import matplotlib.pyplot as plt
import math
import numpy as np
import functools
from matplotlib.animation import FuncAnimation

"""
Dummy Data for Linear Regression
"""
data = [(1, 1), (2, 2), (5, 5.5), (6, 8), (9, 10)]    

"""
Plot the line using theta_values
"""
def plot_line(formula, x_range, order_of_regression, label=None):
    x = np.array(x_range).tolist()  
    y = [formula(update_features(x_i, order_of_regression, 9)) for x_i in x]
    l, = plt.plot(x, y, label=label)
    return l

In [ ]:
"""
Hypothesis Function
"""
def h(x, theta):
    return np.matmul(theta.T, x)[0][0]

"""
Partial Derivative w.r.t. theta_i
"""
def j_prime_theta(data, theta, order_of_regression, i):
    result = 0
    m = len(data)
    for x, y in data :
        x = update_features(x, order_of_regression)
        result += (h(x, theta) - y) * x[i]
    
    return (1/m) * result

"""
Update features by order of the regression
"""

def update_features(x, order_of_regression, m = 9):
    features = [1]
    for i in range(order_of_regression):
        features.append(math.pow(x, i+1)/ math.pow(m, i+1))
    return np.atleast_2d(features).T

"""
Cost Function
"""
def j(data, theta, order_of_regression):
    cost = 0
    m = len(data)
    for x, y in data:
        x = update_features(x, order_of_regression)
        cost += math.pow(h(x, theta) - y, 2)
    return (1/(2*m)) * cost

"""
Simultaneous Update
"""
def update_theta(data, alpha, theta, order_of_regression):
    temp = []
    for i in range(order_of_regression+1):
        temp.append(theta[i] - alpha * j_prime_theta(data, theta, order_of_regression, i))
    theta = np.array(temp)
    return theta
    
"""
Gradient Descent For Multivariate Regression
"""
def gradient_descent(data, alpha, tolerance, theta=[], order_of_regression = 2):
    if len(theta) == 0:
        theta = np.atleast_2d(np.random.random(order_of_regression+1) * 100).T
    prev_j = 10000
    curr_j = j(data, theta, order_of_regression)
    print(curr_j)
    cost_history = []
    theta_history = [] 
    counter = 0
    while(abs(curr_j - prev_j) > tolerance):
        try:
            cost_history.append(curr_j)
            theta_history.append(theta)
            theta = update_theta(data, alpha, theta, order_of_regression)
            prev_j = curr_j
            curr_j = j(data, theta, order_of_regression)
            if counter % 100 == 0:
                print(curr_j)
            counter += 1
        except:
            break
    print("Stopped with Error at %.5f" % prev_j)
    return theta, cost_history, theta_history

theta, cost_history, theta_history = gradient_descent(data, 0.01, 0.0001, order_of_regression=150)

In [ ]:
"""
Regularized Version
"""
def reg_j_prime_theta(data, theta, l, order_of_regression, i):
    result = 0
    m = len(data)
    for x, y in data :
        x = update_features(x, order_of_regression)
        result += (h(x, theta) - y) * x[i]
    result += l*theta[i]
    return (1/m) * result

def reg_j(data, theta, l, order_of_regression):
    cost = 0
    m = len(data)
    for x, y in data:
        x = update_features(x, order_of_regression)
        cost += math.pow(h(x, theta) - y, 2)
    reg = 0
    for j in theta:
        reg += math.pow(j, 2)
    reg = reg * l
    return (1/(2*m)) * (cost + reg)

def reg_update_theta(data, alpha, theta, l, order_of_regression):
    temp = []
    for i in range(order_of_regression+1):
        temp.append(theta[i] - alpha * reg_j_prime_theta(data, theta, l, order_of_regression, i))
    theta = np.array(temp)
    return theta

def reg_gradient_descent(data, alpha, l, tolerance, theta=[], order_of_regression = 2):
    if len(theta) == 0:
        theta = np.atleast_2d(np.random.random(order_of_regression+1) * 100).T
    prev_j = 10000
    curr_j = reg_j(data, theta, l, order_of_regression)
    print(curr_j)
    cost_history = []
    theta_history = [] 
    counter = 0
    while(abs(curr_j - prev_j) > tolerance):
        try:
            cost_history.append(curr_j)
            theta_history.append(theta)
            theta = reg_update_theta(data, alpha, theta, l, order_of_regression)
            prev_j = curr_j
            curr_j = reg_j(data, theta, l, order_of_regression)
            if counter % 100 == 0:
                print(curr_j)
            counter += 1
        except:
            break
    print("Stopped with Error at %.5f" % prev_j)
    return theta, cost_history, theta_history
reg_theta, reg_cost_history, reg_theta_history = reg_gradient_descent(data, 0.01, 1, 0.0001, order_of_regression=150)

In [ ]:
f = functools.partial(h, theta=reg_theta_history[-1])
plt.scatter([i[0] for i in data], [i[1] for i in data])
reg = plot_line(f, (np.array(range(10,91))/10).tolist(), order_of_regression=150, label='regularized')
f = functools.partial(h, theta=theta_history[-1])
unreg = plot_line(f, (np.array(range(10,91))/10).tolist(), order_of_regression=150, label='not regularized')
plt.legend([reg, unreg], ['Regularized', 'Not Regularized'])
plt.title('Effect of Regularization')
plt.show()

In [ ]:
update_features(9,2)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0, 1000)
ax.set_ylim(0, 10000)
b,=plt.plot(range(len(cost_history_01)), cost_history_01, label='alpha=0.01')
c,=plt.plot(range(len(cost_history_001)), cost_history_001, label='alpha=0.001')
d,=plt.plot(range(len(cost_history_0001)), cost_history_0001, label='alpha=0.0001')
e,=plt.plot(range(len(cost_history_00001)), cost_history_00001 ,label='alpha=0.00001')
plt.legend(handles=[b, c, d, e])
plt.title("Effect of learning rate")
ax.set_xlabel('epochs')
ax.set_ylabel('cost')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0, 5000)
ax.set_ylim(0, 6000)
plt.plot(range(len(cost_history)), cost_history)
# plt.title("Optimum learning rate")
ax.set_xlabel('epochs')
ax.set_ylabel('cost')
plt.show()

In [ ]:
from IPython.core.display import display, HTML
fig, ax = plt.subplots()
x = []
y = []
plt.scatter([i[0] for i in data], [i[1] for i in data])
ln, = plt.plot(x, y, 'r--', animated=True, label='h(x)')
plt.title('Parameter Optimization with Feature scaling')

def init():
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 50)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.legend(handles=[ln])
    return ln,

def update(frame):
    theta = frame
    x = (np.array(range(10, 91))/10).tolist()
    f = functools.partial(h, theta=theta)
    y = [f(update_features(x_i, 100)) for x_i in x]
    ln.set_data(x, y)
    return ln,

def data_gen():
    for i in theta_history[::100]:
        yield i

ani = FuncAnimation(fig, update, frames=data_gen(),
                    init_func=init, blit=False, interval=100)
display(HTML(ani.to_html5_video()))

In [ ]:
len(theta_history)

In [ ]:
from IPython.core.display import display, HTML
fig, ax = plt.subplots()
x = []
y = []
plt.scatter([i[0] for i in data], [i[1] for i in data])
ln, = plt.plot(x, y, 'r--', animated=True)
plt.title('Hypothesis Parameter Optimization for Linear Regression')

def init():
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    return ln,

def update(frame):
    theta_0 = frame[0]
    theta_1 = frame[1]
    x = np.array(range(10))
    f = functools.partial(h, theta_0=theta_0, theta_1=theta_1)
    y = f(x)
    ln.set_data(x, y)
    return ln,

def data_gen():
    for i, j in zip(theta_0_history[::300], theta_1_history[::300]):
        yield i, j

ani = FuncAnimation(fig, update, frames=data_gen(),
                    init_func=init, blit=False, interval=100)
display(HTML(ani.to_html5_video()))

In [ ]:
plt.show()